In [ ]:
# =========================
# Environment Setup
# =========================

# Install uv for dependency management
!pip install -q uv
# Retrieve the list of dependencies
!wget https://raw.githubusercontent.com/oliviermeslin/crospint/main/pyproject.toml
# Install dependencies
!uv pip install -r pyproject.toml
# Install crospint
!uv pip install crospint

In [ ]:
# =========================
# The notebook starts here
# =========================

# Modules for data manipulation
import polars as pl
from polars import col as c
import numpy as np

# Modules for machine learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import lightgbm

# Import the model
from crospint.housing_prices import TwoStepsModel, \
    calibrate_model, train_calibration_model

# Import plotting functions
from plotnine import *
from mizani.formatters import percent_format

In [1]:
# Define a function to retrieve the data from S3
def retrieve_transaction_data(type_housing_unit):
    assert type_housing_unit in["houses", "flats"], "type_housing_unit must be either 'houses' or 'flats'"
    df = (
        pl.read_parquet(
            f"https://minio.lab.sspcloud.fr/oliviermeslin/diffusion/DVF/transaction_data_{type_housing_unit}_open_data.parquet"
        )
    )
    return(df)

In [ ]:

# Download data from S3
df_transactions = retrieve_transaction_data("houses")

In [ ]:
df_transactions.head(10)

In [ ]:
# Perform the train-test split
train_val, test = train_test_split(
    df_transactions,
    test_size=0.15,
    random_state=20230516
)

# Perform the train-validation split
train, val = train_test_split(
    train_val,
    train_size=0.80,
    random_state=20230516
)

In [ ]:
# Define model hyperparameters
parameters_price_model = {
    "coord_rotation__coordinates_names": ("x", "y"),
    "coord_rotation__number_axis": 11,
    "date_conversion__date_name": "transaction_date",
    "date_conversion__reference_date": "2014-01-01",
    "model__seed": 20230516,
    "model__n_estimators": 1000,
    "model__learning_rate": 0.2,
    "model__num_leaves": 1023,
    "model__max_depth": 15,
    "model__max_bins": 3000,
    "model__min_child_samples": 75,
    "model__lambda_l2": 20,
    "model__min_gain_to_split": 0.0006,
    "model__bagging_fraction": 1,
    "model__bagging_freq": 0,
    "model__feature_fraction": 0.7
}
target_variable_name = "transaction_amount"
model_features = ['floor_area', 'transaction_date', 'x', 'y', "seashore_distance"]
floor_area_variable = "floor_area"
calibration_variables = ['transaction_year', 'transaction_month', 'id_departement', 'floor_area']

In [ ]:
# Instantiate the model
price_model = TwoStepsModel(
    model=lightgbm.LGBMRegressor(verbose=-1),
    log_transform=True,
    price_sq_meter=True,
    presence_coordinates=True,
    floor_area_name=floor_area_variable
)

In [ ]:
# Pass parameters to the models
price_model.pipe.set_params(
    **parameters_price_model
)

In [ ]:
# Train the model
price_model.fit(
    X=train,
    y=train[target_variable_name].to_numpy().ravel(),
    model_features=model_features,
    X_val=val,
    y_val=val[target_variable_name].to_numpy().ravel(),
    early_stopping_rounds=25
)

In [ ]:
# Calibrate the model
calibration_success = False
convergence_rate = 0
while not calibration_success:
    convergence_rate += 1e-3
    print(convergence_rate)
    price_model, calibration_success = calibrate_model(
        model=price_model,
        X=None, # the validation set is used as calibration set by default
        y=None, # the validation set is used as calibration set by default
        calibration_variables=calibration_variables,
        convergence_rate=convergence_rate,
        max_iter=50,
        verbose=True
    )
print(f"    The model was calibrated with threshold {convergence_rate}")

In [ ]:
# Train the calibration model
price_model = train_calibration_model(
    price_model,
    calibration_model=lightgbm.LGBMRegressor(
        n_estimators=100,
        num_leaves=1023,
        max_depth=12,
        learning_rate=0.5,
        min_child_samples=20,
        max_bins=10000,
        random_state=123456,
        verbose=-1
    ),
    r2_threshold=0.98,
    evaluation_period=5,
    verbose=True
)

In [ ]:
# Compute predictions on the test set
raw_predictions = price_model.predict(
    test,
    add_retransformation_correction=True,
    retransformation_method="Miller"
)
cal_predictions = price_model.predict(
    test,
    add_retransformation_correction=True,
    retransformation_method="calibration"
)

In [ ]:
# Add prediction to the test set
test2 = (
    test
    .with_columns(
        raw_prediction = pl.Series(raw_predictions),
        cal_prediction = pl.Series(cal_predictions)
    )
)

In [ ]:
# Compute metrics on predictive power
r2_raw = r2_score(np.log(test2["transaction_amount"]), np.log(test2["raw_prediction"] / np.exp(price_model.RMSE**2 / 2)))
share_below20_raw = test2.filter((c.raw_prediction / c.transaction_amount - 1).abs() < 0.2).shape[0] / test.shape[0]
r2_cal = r2_score(np.log(test2["transaction_amount"]), np.log(test2["cal_prediction"]))
share_below20_cal = test2.filter((c.cal_prediction / c.transaction_amount - 1).abs() < 0.2).shape[0] / test.shape[0]
print(f"The R² of the model with Miller's correction is {r2_raw: .3f}; {share_below20_raw: .1%} of observations are predicted with an absolute error of less than 20%.")
print(f"The R² of the calibrated model is {r2_cal: .3f}; {share_below20_cal: .1%} of observations are predicted with an absolute error of less than 20%.")

In [ ]:
# Define quantiles
nb_quantiles = 200
probs = np.arange(1, nb_quantiles) / nb_quantiles

# Compute true and predicted quantiles
quantiles = (
    test2.select(
        pl.concat_list([
            pl.struct(
                pl.lit(p).alias("quantile"),
                pl.col("transaction_amount").quantile(p, interpolation="nearest").alias("quantile_true"),
                pl.col("raw_prediction").quantile(p, interpolation="nearest").alias("quantile_raw_prediction"),
                pl.col("cal_prediction").quantile(p, interpolation="nearest").alias("quantile_cal_prediction"),
            )
            for p in probs
        ]).alias("data")
    )
    .explode("data")
    .unnest("data")
    # .with_columns(
    #     pl.format("P{}", (pl.col("percentile") * nb_quantiles).cast(pl.Float32))
    #     .alias("percentile")
    # )
    .select(["quantile", "quantile_true", "quantile_raw_prediction", "quantile_cal_prediction"])
    .with_columns(
        ratio_raw = c.quantile_raw_prediction / c.quantile_true,
        ratio_cal = c.quantile_cal_prediction / c.quantile_true
    )
    .select("quantile", "quantile_true", "ratio_raw", "ratio_cal")
    .unpivot(
       index = ["quantile", "quantile_true"]
    )
    .with_columns(
        model_label = pl.when(c.variable=="ratio_raw").then(pl.lit("Model with Miller's correction")).otherwise(pl.lit("Calibrated model"))
    )
)

In [ ]:
# Plot the calibration curve on the test set
(
    ggplot(quantiles) +
    geom_line(aes(x = "quantile_true", y = "value", color = "model_label"), size = 1) +
    labs(
        x = "True quantile value",
        y = "Ratio predicted quantile/true quantile",
        color = "Model"
    ) +\
    scale_x_log10() +
    scale_color_cmap_d(cmap_name="viridis") +
    theme(legend_position="bottom")
)

In [ ]:
# Compute WMAPB to measure conditional biases
variables_wmapb = ['transaction_year', 'transaction_month', 'id_departement', 'floor_area']
wmapb_results = (
        pl.concat(
        [
            (
                test2
                .group_by(var)
                .agg(
                    nb_observations = pl.len(),
                    bias_raw = (c.raw_prediction.sum() / c.transaction_amount.sum() - 1),
                    bias_cal = (c.cal_prediction.sum() / c.transaction_amount.sum() - 1)
                )
                .select(
                    variable_wmapb = pl.lit(var),
                    wmapb_raw = (c.nb_observations * c.bias_raw.abs()).sum() / c.nb_observations.sum(),
                    wmapb_cal = (c.nb_observations * c.bias_cal.abs()).sum() / c.nb_observations.sum()
                )
            )
            for var in variables_wmapb
        ]
    )
    .unpivot(
       index = ["variable_wmapb"]
    )
    .with_columns(
        model_label = pl.when(c.variable=="wmapb_raw").then(pl.lit("Model with Miller's correction")).otherwise(pl.lit("Calibrated model"))
    )
)

In [ ]:
# Plot WMAPB
(
    ggplot(wmapb_results) +
    geom_bar(
        aes(x = "variable_wmapb", y = "value", fill = "model_label"),
        stat = "identity", position = "dodge"
    ) +
    labs(
        x = "Variable",
        y = "WMAPB metric",
        fill = "Model"
    ) +\
    scale_fill_cmap_d(cmap_name="viridis") +
    scale_y_continuous(labels=percent_format()) +
    theme(legend_position="bottom")
)

In [ ]:
# Time consistency
time_trends = (
    test2
    .group_by("transaction_year")
    .agg(
        ratio_raw = (c.raw_prediction.sum() / c.transaction_amount.sum()),
        ratio_cal = (c.cal_prediction.sum() / c.transaction_amount.sum())
    )
    .unpivot(
       index = ["transaction_year"]
    )
    .with_columns(
        model_label = pl.when(c.variable=="ratio_raw").then(pl.lit("Model with Miller's correction")).otherwise(pl.lit("Calibrated model"))
    )
    .sort("transaction_year")
)

In [ ]:
# Plot time trends
(
    ggplot(time_trends) +
    geom_line(
        aes(x = "transaction_year", y = "value", color = "model_label"), size = 1
    ) +
    labs(
        x = "Year",
        y = "Ratio total predicted amounts /\ntotal observed amounts",
        color = "Model"
    ) +\
    scale_color_cmap_d(cmap_name="viridis") +
    scale_y_continuous(labels=percent_format(), limits = (0.95, 1.05)) +
    theme(legend_position="bottom")
)